In [218]:
import pandas as pd
import numpy as np

# Section 1

In [219]:
def calculate_macd(close_prices):
    short_period=12
    long_period=26
    signal_period=9

    short_ema = close_prices.ewm(span=short_period, adjust=False).mean()
    long_ema = close_prices.ewm(span=long_period, adjust=False).mean()

    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()

    return macd_line, signal_line

In [220]:

def calculate_rsi(close_prices):
    period=14
    delta = close_prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [221]:

def backtest_strategy(close_prices: pd.Series, initial_capital: float):
    # indicators
    macd_line, signal_line = calculate_macd(close_prices)
    rsi = calculate_rsi(close_prices)

    in_position = False  # True means we are in a position, False means no position
    equity = pd.Series(data=[0]*len(close_prices), index=close_prices.index)
    equity.iloc[0] = initial_capital  # Starting with initial capital on all days

    for i in range(1, len(close_prices)):
        # Entry condition: MACD crosses above Signal and RSI > 48
        if (not in_position and macd_line.iloc[i] > signal_line.iloc[i]) and (macd_line.iloc[i - 1] <= signal_line.iloc[i - 1]) and (rsi.iloc[i] > 48):
            in_position = True  # Enter position

        # Exit condition: MACD crosses below Signal
        elif (in_position and macd_line.iloc[i] < signal_line.iloc[i]) and (macd_line.iloc[i - 1] >= signal_line.iloc[i - 1]):
            in_position = False  # Exit position

        # Calculate daily equity
        if in_position:
            equity.iloc[i] = equity.iloc[i - 1] * (1 + ((close_prices.iloc[i] - close_prices.iloc[i-1]) / close_prices.iloc[i-1]))  # Gain or loss if in position
        else:
            equity.iloc[i] = equity.iloc[i - 1]  # Capital remains the same if out of position

    return equity

# Section 2

In [222]:
def calculate_netProfit(equity):
    return equity.iloc[-1] - equity.iloc[0]

In [223]:
def calculate_max_drawdown(equity):
    peak = equity.cummax()
    drawdown = (equity - peak) / peak
    return drawdown.min()*100

In [224]:

def calculate_drawdown_duration(equity_curve):
    cumulative_max = equity_curve.cummax()
    drawdown = equity_curve / cumulative_max - 1
    in_drawdown = drawdown < 0
    drawdown_durations = in_drawdown.astype(int).groupby((in_drawdown != in_drawdown.shift()).cumsum()).cumsum()

    max_duration = drawdown_durations.max()
    return max_duration, drawdown_durations


In [225]:
def calculate_annual_sharp_ratio(equity, N=255, rf=0.04):
    returns = equity.pct_change()
    mean = returns.mean() * N -rf
    sigma = returns.std() * np.sqrt(N)
    return mean / sigma

In [226]:
def calculate_annual_sortino_ratio(equity, N=255,rf=0.04):
    returns = equity.pct_change()
    mean = returns.mean() * N -rf
    std_neg = returns[returns<0].std()*np.sqrt(N)
    return mean/std_neg

# Section 3

In [227]:
import yfinance as yf
import pandas as pd

tickers = {"BTC-USD", "ETH-USD", "DOGE-USD"}

start_date = "2022-10-01"
end_date = "2024-10-01"

crypto_data = {}

# Download data for each cryptocurrency
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    crypto_data[ticker] = data['Adj Close']
Bitcoin_adj_closes = pd.Series(crypto_data['BTC-USD'].values[:,0])
Ethereum_adj_closes = pd.Series(crypto_data['ETH-USD'].values[:,0])
Dogecoin_adj_closes = pd.Series(crypto_data['DOGE-USD'].values[:,0])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [228]:
equity4Bitcoin = backtest_strategy(Bitcoin_adj_closes, 100)
equity4Ethereum = backtest_strategy(Ethereum_adj_closes, 100)
equity4Dogecoin = backtest_strategy(Dogecoin_adj_closes, 100)

<ipython-input-221-21f1641a45fc>:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '101.86804883537121' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  equity.iloc[i] = equity.iloc[i - 1] * (1 + ((close_prices.iloc[i] - close_prices.iloc[i-1]) / close_prices.iloc[i-1]))  # Gain or loss if in position
<ipython-input-221-21f1641a45fc>:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '101.1041057600688' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  equity.iloc[i] = equity.iloc[i - 1] * (1 + ((close_prices.iloc[i] - close_prices.iloc[i-1]) / close_prices.iloc[i-1]))  # Gain or loss if in position
<ipython-input-221-21f1641a45fc>:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of 

In [229]:
def calculate_measures(equity, crypto):
  Net_proit = calculate_netProfit(equity)
  Maximum_drawdown = calculate_max_drawdown(equity)
  Maximum_drawdown_period, Duration_of_drawdown = calculate_drawdown_duration(equity)
  Annual_sharp_ratio = calculate_annual_sharp_ratio(equity)
  Annual_sortino_ratio = calculate_annual_sortino_ratio(equity)
  print(f"Net profit for {crypto} is {Net_proit}$")
  print(f"Maximum drawdown for {crypto} is {Maximum_drawdown}")
  print(f"Duration of drawdown for {crypto} is \n{Duration_of_drawdown.values}")
  print(f"Maximum drawdown period for {crypto} is {Maximum_drawdown_period} days")
  print(f"Annual sharpe ratio for {crypto} is {Annual_sharp_ratio}")
  print(f"Annual sortino ratio for {crypto} is {Annual_sortino_ratio}")

## Bitcoin

In [230]:
calculate_measures(equity4Bitcoin, "Bitcoin")

Net profit for Bitcoin is 218.9784123431886$
Maximum drawdown for Bitcoin is -8.247514008869924
Duration of drawdown for Bitcoin is 
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   1   0   0   1   2   0   1   2   3   4   5   0   0
   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103   0   0   1   0   1
   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24   0   1   0   1   0   1   0   1   2   3   4   5   0
   1   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   1   2
   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18

## Ethereum

In [231]:
calculate_measures(equity4Ethereum, "Ethereum")

Net profit for Ethereum is 404.5220009721855$
Maximum drawdown for Ethereum is -9.53664752493234
Duration of drawdown for Ethereum is 
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   1   0   0   1   2   0   1   2   3   4   5   0   1
   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62   0   1   0   0   0   0   0   0   0   0   0
   1   2   3   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14
  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30   0   1
   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20   0   0   1   2   0   1   2   3   0   1   0   1   2   3   4   5   6
   7   0   1   2   3   0   0   1   2   3   4   0   1   0   0   0   1   0
   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  

## Dogecoin

In [232]:
calculate_measures(equity4Dogecoin, "Dogecoin")

Net profit for Dogecoin is 430.76718965253974$
Maximum drawdown for Dogecoin is -19.944663332826362
Duration of drawdown for Dogecoin is 
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   1   0   0   1   2   3   4   5   6   7   8   9  10  11
  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29
  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
   0   0   1   2   3   4   5   6   7   8   9  10  11  12  1